# 🤖 RAG-система (Retrieval-Augmented Generation)

### Основной принцип RAG

Вместо того чтобы полагаться только на знания, полученные во время обучения (которые могут быть устаревшими или неверными), LLM сначала **извлекает** релевантные факты из внешней, актуальной базы данных (вашего PDF), а затем **генерирует** ответ, используя эти факты как обязательный контекст.

### Этапы конвейера:
1.  **Парсинг и Чанкинг:** Преобразование PDF в текстовые фрагменты.
2.  **Индексация (Векторизация):** Превращение текста в числовые векторы (эмбеддинги).
3.  **Поиск (Retrieval):** Нахождение наиболее семантически близких фрагментов к запросу.
4.  **Сортировка ответов (Rerunking):** Процесс переоценки и переранжирования найденных документов или данных на основе их релевантности запросу.
5.  **Генерация (Augmentation):** Формирование финального ответа LLM на основе найденного контекста.

## 🛠 Шаг 1: Настройка окружения и импорт библиотек

Дополним импорты из прошлого колаба чтобы использовать Vosk (извлечение текста из аудио) и Beautiful Soup (извлечение текста из веб-страницы)

os - Стандартный модуль для взаимодействия с операционной системой.


In [3]:
# 1. Установка зависимостей (для RAG, веб-скрейпинга и Vosk)
!pip install openai numpy pandas requests beautifulsoup4 vosk scipy pydub docling playwright nest-asyncio
!playwright install chromium # Устанавливаем Chromium — браузер, которым будет управлять Playwright для веб-скрейпинга динамических страниц
!playwright install-deps # Устанавливаем системные зависимости, необходимые для запуска Chromium в среде Colab (или другом Linux-окружении)


!pip install --upgrade python-dotenv
# Скачиваем небольшую русскую модель Vosk
!wget -nc https://alphacephei.com/vosk/models/vosk-model-small-ru-0.22.zip
# Распаковываем и переименовываем папку для удобства
!unzip -n vosk-model-small-ru-0.22.zip
!mv vosk-model-small-ru-0.22 model

# 2. Импорт необходимых модулей
import os                      # работа с файловой системой и переменными окружения
import numpy as np             # числовые массивы и вычисления
import pandas as pd            # обработка табличных данных
import requests                # HTTP-запросы (для получения данных с веб-сайтов или API)
import json                    # работа с JSON-данными
import wave                    # чтение/запись WAV-аудиофайлов
import tempfile                # создание временных файлов (например, для обработки аудио)
import asyncio                 # асинхронное программирование
import nest_asyncio            # позволяет запускать asyncio в Jupyter/Colab
from bs4 import BeautifulSoup  # парсинг HTML/XML для извлечения текста
from io import BytesIO         # работа с байтовыми потоками (например, для обработки загруженных файлов в памяти)
from openai import OpenAI      # клиент для OpenAI API (включая использование embedding и чат-моделей)
from vosk import Model, KaldiRecognizer  # офлайн-распознавание речи через Vosk
from pydub import AudioSegment # конвертация аудиоформатов (например, mp3 → wav, изменение частоты дискретизации)
from google.colab import files # позволяет загружать файлы в ноутбук
from docling.document_converter import DocumentConverter  # извлечение текста из PDF/DOCX и др.
from playwright.async_api import async_playwright  # асинхронная автоматизация браузера для веб-скрейпинга

print("Зависимости установлены и переменные инициализированы.")

Installing dependencies...
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://cli.github.com/packages stable InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entr

/usr/local/lib/python3.12/dist-packages/pydub/utils.py:300: SyntaxWarning: invalid escape sequence '\('
  m = re.match('([su]([0-9]{1,2})p?) \(([0-9]{1,2}) bit\)$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:301: SyntaxWarning: invalid escape sequence '\('
  m2 = re.match('([su]([0-9]{1,2})p?)( \(default\))?$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:310: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(flt)p?( \(default\))?$', token):
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:314: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(dbl)p?( \(default\))?$', token):


Зависимости установлены и переменные инициализированы.


## 📂 Шаг 2: Загрузка файла

Загрузим необходимый файл (PDF или wav), который послужит нашей внешней базой знаний (корпусом).

**Инструкция:** Запустите ячейку и выберите файл на вашем устройстве.

In [4]:
# Блок загрузки для PDF
print("Начало загрузки файла...")
uploaded = files.upload()

# Сохраняем имя файла для дальнейшей работы

#Берем первый файл
pdf_path = list(uploaded.keys())[0]

print(f"✅ Файл '{pdf_path}' успешно загружен.")

Начало загрузки файла...


Saving instrukciya-samsung-galaxy-s23-ultra-6.pdf to instrukciya-samsung-galaxy-s23-ultra-6.pdf
✅ Файл 'instrukciya-samsung-galaxy-s23-ultra-6.pdf' успешно загружен.


In [ ]:
# Блок загрузки для wav
print("Начало загрузки файла...")
uploaded = files.upload()

# Сохраняем имя файла для дальнейшей работы

#Берем первый файл
vosk_audio = list(uploaded.keys())[0]

print(f"✅ Файл '{vosk_audio}' успешно загружен.")

## 📝 Шаг 3: Предварительная обработка данных

### 3.1 Парсинг PDF документа

Функция `parse_pdf_document` отвечает за извлечение структурированного текста из PDF.
Мы проходим по каждой странице (`doc.load_page(page_num)`) и используем метод `page.get_text()`, который преобразует элементы PDF в чистый, пригодный для анализа текст.

In [5]:
# Функция извлечения текста из PDF
def parse_pdf_document(file_path: str) -> str:

# Извлекаем текст из PDF-файла с помощью Docling.
    try:
        converter = DocumentConverter()
        result = converter.convert(file_path)
        return result.document.export_to_text()
    except Exception as e:
        print(f"Ошибка при обработке PDF: {e}")
        return ""

if uploaded:
    filename = list(uploaded.keys())[0]  # Берём первый (и, скорее всего, единственный) файл
    print(f"\nОбработка файла: {filename}")

    document_text = parse_pdf_document(filename)
    print(f"\n✅ Извлечено {len(document_text)} символов.\n")

    # Опционально: вывести первые 500 символов для проверки
    print("Превью текста:")
    print(document_text[:500] + ("..." if len(document_text) > 500 else ""))
else:
    print("Файл не был загружен.")


Обработка файла: instrukciya-samsung-galaxy-s23-ultra-6.pdf


[INFO] 2025-12-03 19:59:36,842 [RapidOCR] base.py:22: Using engine_name: torch
[INFO] 2025-12-03 19:59:36,849 [RapidOCR] download_file.py:68: Initiating download: https://www.modelscope.cn/models/RapidAI/RapidOCR/resolve/v3.4.0/torch/PP-OCRv4/det/ch_PP-OCRv4_det_infer.pth
[INFO] 2025-12-03 19:59:38,207 [RapidOCR] download_file.py:82: Download size: 13.83MB
[INFO] 2025-12-03 19:59:38,390 [RapidOCR] download_file.py:95: Successfully saved to: /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_PP-OCRv4_det_infer.pth
[INFO] 2025-12-03 19:59:38,392 [RapidOCR] torch.py:54: Using /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_PP-OCRv4_det_infer.pth
[INFO] 2025-12-03 19:59:38,660 [RapidOCR] base.py:22: Using engine_name: torch
[INFO] 2025-12-03 19:59:38,662 [RapidOCR] download_file.py:68: Initiating download: https://www.modelscope.cn/models/RapidAI/RapidOCR/resolve/v3.4.0/torch/PP-OCRv4/cls/ch_ptocr_mobile_v2.0_cls_infer.pth
[INFO] 2025-12-03 19:59:39,844 [RapidOCR] downlo


✅ Извлечено 275789 символов.

Превью текста:
## РУКОВОДСТВО ПОЛЬЗОВАТЕЛЯ

SM-S911B/DS SM-S916B/DS

SM-S918B/DS

## Содержание

## Начало работы

5

Внешний вид устройства и функции

11

16

18

19

20

21

23

31

33

34

Зарядка аккумулятора

Карта

nano-SIM

и eSIM

Включение и выключение устройства

Начальная настройка

Samsung account

Перенос данных с предыдущего устройства

(Smart Switch)

Описание функций экрана

Панель уведомлений

Снимок экрана и запись действий на экране

Ввод текста

## Приложения и функции

37

Установка и удал...


### 3.2 Парсинг аудио

Функция `parse_audio_data` отвечает за извлечение структурированного текста из wav аудио.

In [ ]:
vosk_model_path = 'vosk-model-folder'

def parse_audio_data(file_path):
    if not os.path.exists(vosk_model_path):
        return "Error: Model not found"

    try:
        # Предобработка Аудио (pydub)
        audio = AudioSegment.from_file(file_path)v1
        # Конвертирует аудио в формат, необходимый для Vosk: моно (.set_channels(1)) с частотой дискретизации 16 кГц (.set_frame_rate(16000))
        audio = audio.set_frame_rate(16000).set_channels(1)
        #Сохраняет (перезаписывает) измененный аудиофайл в формате WAV по тому же пути
        audio.export(file_path, format='wav')

        # Загрузка WAV при помощи wave
        wf = wave.open(file_path, 'rb')
        # Создает объект Model, загружая русскую модель Vosk
        model = Model(vosk_model_path)
        # Создает объект KaldiRecognizer, который является основным инструментом для распознавания
        rec = KaldiRecognizer(model, wf.getframerate())
        # Указывает Vosk не возвращать информацию о времени начала и конца каждого слова, а только сплошной текст
        rec.SetWords(False)

        # Поточное распознавание
        full_text = []
        while True:
            # Читает аудиофайл порциями
            data = wf.readframes(4000)
            if len(data) == 0:
                break
            # Отправляет каждую порцию в распознаватель (rec.AcceptWaveform(data))
            if rec.AcceptWaveform(data):
                # Если Vosk накопил достаточно данных, он возвращает промежуточный результат (rec.Result()), который парсится из JSON
                result = json.loads(rec.Result())
                # И распознанный текст добавляется в список full_text
                full_text.append(result.get('text', ''))

        # После окончания цикла (чтения всех фреймов) вызывается rec.FinalResult() для получения текста, который остался в буфере распознавателя.
        result_final = json.loads(rec.FinalResult())
        full_text.append(result_final.get('text', ''))

        # Закрывает WAV-файл
        wf.close()
        #  Объединяет все сегменты текста из списка full_text в одну строку, разделяя пробелами, и возвращает конечный распознанный текст
        return ' '.join(full_text).strip()

    # Блок try...except перехватывает любые другие ошибки, которые могут возникнуть и возвращает сообщение об ошибке
    except Exception as e:
        return f'Error: {str(e)}'

In [ ]:
# Запуск парсинга

print(f"\n-> Обработка АУДИО: {vosk_audio} (Транскрипция Vosk)")
audio_text = parse_audio_data(vosk_audio)

print("\n" + "*"*40)
print(f"ИЗВЛЕЧЕННЫЙ ТЕКСТ (первые 800 символов):\n{audio_text[:800]}...")
print("*"*40)


-> Обработка АУДИО: temp_audio_news (1).wav (Транскрипция Vosk)

****************************************
ИЗВЛЕЧЕННЫЙ ТЕКСТ (первые 800 символов):
привет меня зовут евгений коваленко в этом видео я расскажу о самых свежих новостях из мира робототехники от четырёх впечатляющих гуманоид ных роботах который только недавно вышли на сцену начнём сью не три джой один от китайская компания июне три работе кс это компактный робот он весит всего тридцать пять килограмм но на недавней демонстрации от бензина кедами о фортификации о интеллидженс он без труда потянул за собой белый седан массой в почти в полторы тонны это в почти в пятьдесят раз больше его собственного веса он обучен по алгоритму сор меня лично есть небольшие вопросы к назад да этого со этого тора то есть получается у них есть и да у них есть эйч у них есть и дабл ю и то есть но в середине вы проигнорирует название и поищем где тут найти букву о вот она да и таким образом у нас ...
****************************************


### 3.2 Парсинг веб-страницы

Функция `parse_website_data` отвечает за извлечение структурированного текста из веб-страницы.

In [36]:
web_text = ""

nest_asyncio.apply()

async def _fetch_html(url: str) -> str:
    playwright = await async_playwright().start()
    browser = await playwright.chromium.launch(
        headless=True,
        args=["--no-sandbox", "--disable-gpu", "--disable-dev-shm-usage"]
    )
    page = await browser.new_page()

    # Блокируем ненужные ресурсы
    await page.route("**/*", lambda route: route.abort() if route.request.resource_type in ["image", "stylesheet", "font"] else route.continue_())

    page.set_default_timeout(60000)
    await page.goto(url, wait_until="domcontentloaded")

    # Ждём хотя бы заголовок
    try:
        await page.wait_for_selector("h1", timeout=20000)
    except:
        pass  # не критично

    html = await page.content()
    await browser.close()
    await playwright.stop()
    return html

def parse_website_data(url: str) -> str:
    url = url.strip()
    try:
        print("Шаг 1: Загружаем страницу...")
        loop = asyncio.get_event_loop()
        html_content = loop.run_until_complete(_fetch_html(url))
        print("HTML получен!")

        print("Шаг 2: Сохраняем во временный файл...")
        with tempfile.NamedTemporaryFile(mode="w", suffix=".html", delete=False, encoding="utf-8") as tmp:
            tmp.write(html_content)
            temp_path = tmp.name

        print("Шаг 3: Извлекаем текст через Docling...")
        converter = DocumentConverter()
        result = converter.convert(temp_path)
        clean_text = result.document.export_to_text()
        os.remove(temp_path)

        print(f"Успешно! {len(clean_text)} символов.")
        print("--------------------------\n")
        return clean_text

    except Exception as e:
        print(f"\n ОШИБКА: {e}")
        return ""

In [37]:
# Запуск парсинга

WEBSITE_URL = "https://habr.com/ru/articles/844450/"

print(f"-> Обработка ВЕБ-САЙТА: {WEBSITE_URL}")
clean_text = parse_website_data(WEBSITE_URL)

print("\n" + "*"*40)
print(f"ИЗВЛЕЧЕННЫЙ ТЕКСТ (первые 800 символов):\n{clean_text[:800]}...")
print("*"*40)

-> Обработка ВЕБ-САЙТА: https://habr.com/ru/articles/844450/
🔍 Шаг 1: Загружаем страницу (с оптимизациями)...
✅ HTML получен!
📄 Шаг 2: Сохраняем во временный файл...
🧠 Шаг 3: Извлекаем текст через Docling...


✅ Успешно! 19459 символов.
--------------------------


****************************************
ИЗВЛЕЧЕННЫЙ ТЕКСТ (первые 800 символов):
# Основы Whitespace: невидимого языка программирования

Уровень сложности Простой Время на прочтение 7 мин Охват и читатели 9.6K [Ненормальное программирование *](/ru/hubs/crazydev) [Программирование *](/ru/hubs/programming) FAQ Перевод

[Автор оригинала: Script Kiddie](https://www.youtube.com/watch?v=ea7HZ96dwqs&ab_channel=ScriptKiddie)

Привет, Хабр! Сегодня речь пойдет про whitespace - язык программирования, который использует в качестве своего алфавита только 3 непечатных символа.

Этот *эзотерический* язык был создан Эдвином Брэди и Крисом Моррисом в 2003 и, разумеется, не предназначен для практического применения, существуя как определенный вызов для программистов - мол, попробуй отладить невидимый код.

Итак, как вы уже поняли, основная особенность whitespace - это то, что весь его ...
****************************************


### 3.3 Чанкинг (Разбиение) по Скользящему Окну

Крупные LLM имеют ограничения на размер входного контекста. Для эффективного поиска мы должны разбить документ на управляемые фрагменты (**чанки**).

Функция `create_sliding_window_chunks` использует технику **Скользящего Окна (Sliding Window)**:

* **`max_chars` (Размер окна):** Определяет максимальную длину одного фрагмента (по умолчанию 1000 символов). Этот размер должен быть оптимизирован под возможности модели эмбеддинга и LLM.
* **`stride_overlap` (Шаг/Перекрытие):** Определяет, сколько символов из предыдущего фрагмента должно быть включено в следующий (по умолчанию 200 символов).

> **Важность Перекрытия (Overlap):** Перекрытие критически важно для сохранения **семантической непрерывности**. Если ключевое предложение или идея находятся на границе двух фрагментов, перекрытие гарантирует, что эта идея будет полностью захвачена хотя бы в одном из них, что повышает точность поиска (Retrieval).

Механизм перекрытия:
Между фрагментами есть область дублирования размером stride:
1. Фрагмент 1: [0 ... 1000] → Конец 1 в позиции 1000
2. Фрагмент 2: [800 ... 1800] → Начало 2 в позиции 800
3. Область [800 1000] (200 символов) входит в оба фрагмента → это перекрытие.

In [ ]:
def create_sliding_window_chunks(document_text, max_chars=1000, stride_overlap=200):
    """Разбивает текст на фрагменты, используя скользящее окно."""
    corpus_fragments = []
    # Шаг, на который сдвигается окно (Размер - Перекрытие)
    stride = max_chars - stride_overlap

    if stride <= 0:
        raise ValueError("Размер перекрытия должен быть меньше размера фрагмента.")

    # Создаём последовательность начальных позиций для разрезания текста с шагом stride.
    for start_index in range(0, len(document_text), stride):
        # Окно заканчивается на start_index + max_chars
        end_index = min(start_index + max_chars, len(document_text))
        fragment = document_text[start_index:end_index]
        corpus_fragments.append(fragment)

        # Если конец фрагмента совпал с концом текста, останавливаемся
        if end_index == len(document_text):
            break

    return corpus_fragments

### 3.4 Применяем функцию для данных полученных из разных источников

In [ ]:
# PDF-документ
corpus_fragments_pdf = create_sliding_window_chunks(document_text, max_chars=1000, stride_overlap=200)
print(f"Документ разбит на {len(corpus_fragments_pdf)} фрагментов для индексации.")

Документ разбит на 343 фрагментов для индексации.


In [ ]:
# Аудио
corpus_fragments_audio = create_sliding_window_chunks(audio_text, max_chars=500, stride_overlap=100)
print(f"Документ разбит на {len(corpus_fragments_audio)} фрагментов для индексации.")

Документ разбит на 17 фрагментов для индексации.


In [ ]:
# Сайт
corpus_fragments_web = create_sliding_window_chunks(web_text, max_chars=250, stride_overlap=50)
print(f"Документ разбит на {len(corpus_fragments_web)} фрагментов для индексации.")

Документ разбит на 13 фрагментов для индексации.


## 🧠 Шаг 4: Индексация (Векторизация)

### 4.1 Эмбеддинги и Векторное Пространство

На этом этапе текст преобразуется в **эмбеддинги** — многомерные числовые векторы (например, из 768 или 1024 чисел). Модель эмбеддинга (здесь `BAAI/bge-multilingual-gemma2`) обучена так, чтобы тексты с похожим **смыслом (семантикой)** имели векторы, близкие друг к другу в этом пространстве. Это позволяет искать по *смыслу*, а не по *совпадению ключевых слов*.

**Модель:** `BAAI/bge-multilingual-gemma2` выбрана как высокоэффективная модель, поддерживающая множество языков (включая русский), что важно для работы с русскоязычными документами и запросами.

In [ ]:
# 🔑 Установка API ключа. Получите его в Nebius Studio.
os.environ["OPENAI_API_KEY"] = "v1.CmMKHHN0YXRpY2tleS1lMDBjejdzOGc2MXA5eXhreWgSIXNlcnZpY2VhY2NvdW50LWUwMHYyeTU5NG5oM2hkN3B0ajIMCNr-nMkGEOiKs6YCOgsI2oG1lAcQgIGuNkACWgNlMDA.AAAAAAAAAAEJ-dDCr2P3_uyCxxqjqE6Rm2CPaDU1SDa9JMhRVazt8XAu5NsdgUzF9r0O9e62BkNtVIgbN-c0w1ovRT9j1TEG"

# Настройка клиента для Nebius AI (совместим с OpenAI API)
client = OpenAI(
    base_url="https://api.studio.nebius.com/v1/",
    api_key=os.getenv("OPENAI_API_KEY")
)

def create_embeddings(text_list, model="BAAI/bge-multilingual-gemma2"):
    """Вызывает API для преобразования списка текстов в список векторов (эмбеддингов)."""
    # Вызов метода embeddings.create для получения векторов
    response = client.embeddings.create(input=text_list, model=model)
    return response.data

In [ ]:
# PDF-документ
# Индексация: Генерация векторов для всех фрагментов корпуса
print("⏳ Генерация эмбеддингов для всего корпуса... (требует времени)")
fragment_embeddings_pdf = create_embeddings(corpus_fragments_pdf)
print(f"✅ Индексация завершена. Получено {len(fragment_embeddings_pdf)} векторов.")

⏳ Генерация эмбеддингов для всего корпуса... (требует времени)
✅ Индексация завершена. Получено 343 векторов.


In [ ]:
# Аудио
# Индексация: Генерация векторов для всех фрагментов корпуса
print("⏳ Генерация эмбеддингов для всего корпуса... (требует времени)")
fragment_embeddings_audio = create_embeddings(corpus_fragments_audio)
print(f"✅ Индексация завершена. Получено {len(fragment_embeddings_audio)} векторов.")

⏳ Генерация эмбеддингов для всего корпуса... (требует времени)
✅ Индексация завершена. Получено 17 векторов.


In [ ]:
# Сайт
# Индексация: Генерация векторов для всех фрагментов корпуса
print("⏳ Генерация эмбеддингов для всего корпуса... (требует времени)")
fragment_embeddings_web = create_embeddings(corpus_fragments_web)
print(f"✅ Индексация завершена. Получено {len(fragment_embeddings_web)} векторов.")

⏳ Генерация эмбеддингов для всего корпуса... (требует времени)
✅ Индексация завершена. Получено 13 векторов.


## 🔍 Шаг 5: Извлечение контекста (Retrieval)

Когда пользователь задает вопрос, мы должны найти наиболее релевантные фрагменты.

### 5.1 Косинусное сходство (Cosine Similarity)

Это метрика, используемая для измерения сходства между векторами. Она рассчитывает косинус угла $\theta$ между двумя векторами $A$ и $B$:

$$\text{similarity} = \cos(\theta) = \frac{A \cdot B}{\|A\| \cdot \|B\|}$$

* Если $\text{similarity} = 1$, векторы полностью совпадают по направлению (максимальное сходство).
* Если $\text{similarity} = 0$, векторы ортогональны (независимы, нет сходства).
* Если $\text{similarity} = -1$, векторы противоположны по направлению (максимальное различие).

Векторы запроса и фрагментов с максимальным косинусным сходством будут считаться наиболее **семантически** релевантными.

In [ ]:
def calculate_cosine_similarity(vec1, vec2):
    """Вычисляет косинусное сходство между двумя векторами NumPy."""
    # np.dot(vec1, vec2) - скалярное произведение
    # np.linalg.norm(vec) - Евклидова норма (длина) вектора
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

def retrieve_context(user_query, corpus_fragments, fragment_embeddings, k_neighbors=3):
    """Находит топ-K фрагментов корпуса, наиболее близких к запросу по смыслу.

    :param k_neighbors: Количество самых релевантных фрагментов для извлечения.
    :return: Список из K фрагментов текста.
    """
    # 1. Векторизация запроса
    query_embedding_response = client.embeddings.create(
        input=[user_query],
        model="BAAI/bge-multilingual-gemma2"
    )
    # Преобразуем список чисел в вектор NumPy для расчетов
    query_embedding = np.array(query_embedding_response.data[0].embedding)

    # 2. Расчет сходства
    similarity_scores = []
    for i, emb_data in enumerate(fragment_embeddings):
        fragment_vector = np.array(emb_data.embedding)
        score = calculate_cosine_similarity(query_embedding, fragment_vector)
        similarity_scores.append((i, score))

    # 3. Сортировка и выбор топ-K
    # Сортируем по убыванию (чем выше score, тем лучше)
    similarity_scores.sort(key=lambda x: x[1], reverse=True)
    top_k_indices = [index for index, score in similarity_scores[:k_neighbors]]

    return [corpus_fragments[i] for i in top_k_indices]

## 💬 Шаг 6: Генерация ответа (Augmented Generation)

На этом этапе LLM получает **Полный Промпт**, который состоит из трех частей:

1.  **Системная Инструкция (`system_prompt`):** Определяет роль модели (например, «Ты — ассистент по документам»). Устанавливает жесткое правило: отвечать **ТОЛЬКО** на основе предоставленного контекста.
2.  **Контекст (`context_fragments`):** Те самые $K$ релевантных фрагментов, найденных на предыдущем шаге.
3.  **Запрос Пользователя (`query`):** Оригинальный вопрос.

### Параметры Генерации

* **`model`:** Используется мощная модель, например, `Qwen/Qwen3.5-32B`.
* **`temperature=0`:** Этот параметр устанавливает минимальный уровень случайности (креативности) в ответах. Установка `0` гарантирует, что модель будет стремиться к **детерминированному, фактическому** ответу на основе предоставленного контекста, что является критически важным требованием для RAG-систем.

In [ ]:
def generate_augmented_response(query, context_fragments, model_name="Qwen/Qwen3-32B"):
    """Формирует промпт из контекста и запроса и вызывает LLM для ответа."""

    system_prompt = "Вы — экспертный ассистент по работе с документами. Ваш ответ должен быть строго основан на предоставленном КОНТЕКСТЕ. Если в КОНТЕКСТЕ нет информации для полного ответа, вы должны сообщить, что информация не найдена, не выдумывая фактов."

    # Объединяем найденные фрагменты в единый, легко читаемый контекст
    context_text = "\n\n--- РЕЛЕВАНТНЫЙ ФРАГМЕНТ ---\n\n".join(context_fragments)

    full_user_message = f"""
    КОНТЕКСТ ДЛЯ ОТВЕТА:
    {context_text}

    ВОПРОС ПОЛЬЗОВАТЕЛЯ:
    {query}
    """

    response = client.chat.completions.create(
        model=model_name,
        temperature=0, # Обеспечивает фактическую точность и снижает "галлюцинации"
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": full_user_message}
        ]
    )

    return response.choices[0].message.content

## 🚀 Шаг 7: Тестирование системы

Запустите ячейку ниже для интерактивного тестирования. Введите вопрос, и система выполнит полный цикл RAG: векторизацию запроса, поиск в корпусе и генерацию ответа LLM.

In [ ]:
# Сначала можем посмотреть как LLM ответит на наш запрос без работы RAG.
# Пример запроса "Что ты знаешь про Figure 02?"
# Интерактивный ввод вопроса
user_query = input("Введите ваш вопрос: ")

# Генерируем финальный ответ
response = client.chat.completions.create(
    model="Qwen/Qwen3-32B",
    temperature=0, # Обеспечивает фактическую точность и снижает "галлюцинации"
    messages=[
        {"role": "user", "content": user_query}
    ]
)

final_answer = response.choices[0].message.content

print("\n" + "="*60)
print("ФИНАЛЬНЫЙ ОТВЕТ СИСТЕМЫ RAG:")
print("="*60)
print(final_answer)
print("="*60)

Введите ваш вопрос: Что ты знаешь про Figure 02?

ФИНАЛЬНЫЙ ОТВЕТ СИСТЕМЫ RAG:
<think>
Хорошо, пользователь спрашивает о Figure 02. Сначала мне нужно понять, о чем именно идет речь. Figure 02 может относиться к разным вещам. Возможно, это название компании, продукта, проекта или что-то еще. Начну с того, что вспомню известные мне компании или проекты с похожими названиями.

Первое, что приходит в голову, это компания Figure, которая занимается разработкой гуманоидных роботов. Возможно, Figure 02 — это их второй прототип или версия робота. Ранее они представили Figure 01, и логично предположить, что Figure 02 — это обновленная версия. Нужно проверить, есть ли информация о Figure 02 в контексте Figure Robotics.

Также стоит подумать о других возможных интерпретациях. Например, в академических статьях или отчетах часто используются обозначения вроде Figure 02 для ссылок на рисунки. Но в данном случае, учитывая, что пользователь спрашивает "Что ты знаешь", скорее всего, имеется в виду конк

In [ ]:
# PDF
# Пример запроса "Как открыть камеру?"
# Интерактивный ввод вопроса
user_query = input("Введите ваш вопрос по загруженному файлу: ")

print("\n" + "*"*40)
print("🔍 Этап 1: Извлечение контекста (Retrieval)")

# Извлекаем 3 самых релевантных фрагмента
top_k_fragments = retrieve_context(user_query, corpus_fragments_pdf, fragment_embeddings_pdf, k_neighbors=3)

# Выводим найденные фрагменты для проверки прозрачности работы
for i, chunk in enumerate(top_k_fragments):
    print(f"\n--- Найденный фрагмент {i+1} ---\n{chunk[:200]}...") # Выводим только первые 200 символов

print("\n" + "*"*40)
print("🤖 Этап 2: Генерация ответа (Augmentation)")

# Генерируем финальный ответ
final_answer = generate_augmented_response(user_query, top_k_fragments)

print("\n" + "="*60)
print("ФИНАЛЬНЫЙ ОТВЕТ СИСТЕМЫ RAG:")
print("="*60)
print(final_answer)
print("="*60)

Введите ваш вопрос по загруженному файлу: Как открыть камеру?

****************************************
🔍 Этап 1: Извлечение контекста (Retrieval)

--- Найденный фрагмент 1 ---
итесь переключателя 
Категории бесед, чтобы включить соответствующую функцию.
Удаление сообщений
Коснитесь сообщения, которое нужно удалить, и удерживая его выберите пункт Удалить.
Изменение настроек ...

--- Найденный фрагмент 2 ---
 
съемки.
•	 Геотеги: добавление тега GPS к снимку.
•	 Сила GPS-сигнала может снизиться, если устройство используется в местах, где прием 
сигнала затруднен, например между зданиями или в низинах, а т...

--- Найденный фрагмент 3 ---

значка 
 влево на заблокированном экране.
•	 Некоторые функции камеры недоступны при открытии приложения Камера с экрана блокировки 
или при выключенном экране, если выбран способ блокировки экрана.
...

****************************************
🤖 Этап 2: Генерация ответа (Augmentation)

ФИНАЛЬНЫЙ ОТВЕТ СИСТЕМЫ RAG:
<think>
Хорошо, пользователь спрашива

In [ ]:
# Аудио
# Пример запроса "Расскажи про робота XPeng Iron"
# Интерактивный ввод вопроса
user_query = input("Введите ваш вопрос по загруженному файлу: ")

print("\n" + "*"*40)
print("🔍 Этап 1: Извлечение контекста (Retrieval)")

# Извлекаем 3 самых релевантных фрагмента
top_k_fragments = retrieve_context(user_query, corpus_fragments_audio, fragment_embeddings_audio, k_neighbors=3)

# Выводим найденные фрагменты для проверки прозрачности работы
for i, chunk in enumerate(top_k_fragments):
    print(f"\n--- Найденный фрагмент {i+1} ---\n{chunk[:200]}...") # Выводим только первые 200 символов

print("\n" + "*"*40)
print("🤖 Этап 2: Генерация ответа (Augmentation)")

# Генерируем финальный ответ
final_answer = generate_augmented_response(user_query, top_k_fragments)

print("\n" + "="*60)
print("ФИНАЛЬНЫЙ ОТВЕТ СИСТЕМЫ RAG:")
print("="*60)
print(final_answer)
print("="*60)

Введите ваш вопрос по загруженному файлу: Расскажи про робота XPeng Iron

****************************************
🔍 Этап 1: Извлечение контекста (Retrieval)

--- Найденный фрагмент 1 ---
 скоро такие аватары будут в каждом доме у кого есть конечно такая возможность приобрести их все ссылки на все упомянутые роботы ресурсы я оставлю в описании под этим видео вы можете под держать это в...

--- Найденный фрагмент 2 ---
шинам находятся в нейтральном положении да то есть на не травке но координация всего тела в реальном времени поражает он адаптируется на ходу как настоящий атлет это не просто трюк а данный робот сон ...

--- Найденный фрагмент 3 ---
амыми близкими к человеческим рукам для невероятной ловкостью он может складывать вещи он может отвечать на вопросы он может проводить экскурсии или даже может работать как гид питается робот твердоте...

****************************************
🤖 Этап 2: Генерация ответа (Augmentation)

ФИНАЛЬНЫЙ ОТВЕТ СИСТЕМЫ RAG:
<think>
Хорошо, пользовате

In [ ]:
# Сайт
# Пример запроса "Что ты знаешь про Figure 02?"
# Интерактивный ввод вопроса
user_query = input("Введите ваш вопрос по загруженному файлу: ")

print("\n" + "*"*40)
print("🔍 Этап 1: Извлечение контекста (Retrieval)")

# Извлекаем 3 самых релевантных фрагмента
top_k_fragments = retrieve_context(user_query, corpus_fragments_web, fragment_embeddings_web, k_neighbors=1)

# Выводим найденные фрагменты для проверки прозрачности работы
for i, chunk in enumerate(top_k_fragments):
    print(f"\n--- Найденный фрагмент {i+1} ---\n{chunk[:200]}...") # Выводим только первые 200 символов

print("\n" + "*"*40)
print("🤖 Этап 2: Генерация ответа (Augmentation)")

# Генерируем финальный ответ
final_answer = generate_augmented_response(user_query, top_k_fragments)

print("\n" + "="*60)
print("ФИНАЛЬНЫЙ ОТВЕТ СИСТЕМЫ RAG:")
print("="*60)
print(final_answer)
print("="*60)

Введите ваш вопрос по загруженному файлу: Что ты знаешь про Figure 02?

****************************************
🔍 Этап 1: Извлечение контекста (Retrieval)

--- Найденный фрагмент 1 ---
оботы стали быстрее и лучше справлялись с задачами на производстве.
figure ai
figure
f.02
figure 02
роботы
bmw
гуманоидные роботы
человекоподобные роботы
Робототехника
Транспорт
Будущее здесь
Развитие...

****************************************
🤖 Этап 2: Генерация ответа (Augmentation)

ФИНАЛЬНЫЙ ОТВЕТ СИСТЕМЫ RAG:
<think>
Хорошо, пользователь спрашивает о Figure 02. Начну с анализа предоставленного контекста. Вижу, что в контексте упоминаются "роботы", "гуманоидные роботы", "человекоподобные роботы", "Робототехника", "Транспорт", "Будущее здесь", "Развитие стартапа", "Информационная служба Хабра". Также есть "figure ai", "figure", "f.02", "figure 02". 

Возможно, Figure 02 — это модель или версия робота от компании Figure AI. В контексте говорится, что роботы стали быстрее и лучше справляться с задача